In [ ]:
from sage.repl.image import Image as sageImage
from PIL import Image as PILImage

def imagetomatrices(sageimage):
    R = matrix(RDF, sageimage.width(), sageimage.height(), lambda i, j: sageimage.pixels()[i,j][0])
    G = matrix(RDF, sageimage.width(), sageimage.height(), lambda i, j: sageimage.pixels()[i,j][1])
    B = matrix(RDF, sageimage.width(), sageimage.height(), lambda i, j: sageimage.pixels()[i,j][2])
    
    return [R,G,B]

def matricestoimage(matrices, filename='untitled.png', saveimage=False, showimage=False):
    [Rrows, Rcolumns] = matrices[0].dimensions()
    [Grows, Gcolumns] = matrices[1].dimensions()
    [Brows, Bcolumns] = matrices[2].dimensions()
    
    if Rrows != Grows or Grows != Brows or Rcolumns != Gcolumns or Gcolumns != Bcolumns:
        raise Exception('Matrices must be of equal dimensions')
        
    sageimage = sageImage('RGB', (Rcolumns, Rrows))
    pixels = sageimage.pixels()
    for i in range(Rcolumns):
        for j in range(Rrows):
            rgbtuple = tuple([(matrices[0][j,i]).round(), (matrices[1][j,i]).round(), (matrices[2][j,i]).round()])
            pixels[i,j] = rgbtuple
    if saveimage:
        sageimage.save(filename)
    if showimage:
        sageimage.show()
    return sageimage

def filetomatrices(filename):
    PILimage = PILImage.open(filename)
    listdata = list(PILimage.getdata())
    [width, height] = PILimage.size
    
    R = matrix(RDF, height, width, [rgbtriple[0] for rgbtriple in listdata])
    B = matrix(RDF, height, width, [rgbtriple[1] for rgbtriple in listdata])
    G = matrix(RDF, height, width, [rgbtriple[2] for rgbtriple in listdata])
    
    return [R,B,G]

def compressmatrix(matrix, singularvalues):
    [columns, rows] = matrix.dimensions()
    if singularvalues > min(columns, rows):
        raise Exception('Number of singular values exceeded')
        
    [U,S,V] = matrix.SVD()
    
    return U[:,:singularvalues] * S[:singularvalues, :singularvalues] * (V[:,:singularvalues]).conjugate_transpose()

def compressimage(image, k, filename='untitled_compressed.png', showresult=False, saveresult=False):
    if type(image) == str:
        matrices = filetomatrices(image)
        
    elif type(image) == sage.repl.image.Image:
        matrices = imagetomatrices(image)
        
    elif type(image) == list and len(image) == 3:
        matrices = image
    
    rankkmatrices = [compressmatrix(matrices[0],k), compressmatrix(matrices[1],k), compressmatrix(matrices[2],k)]
    
    return matricestoimage(rankkmatrices, filename, showresult, saveresult)

# Gebrauchsanweisung

## Von einem Bild ins Programm nach Matrizen

Synopsis:

    imagetometrices(sageimage)
    
    sageimage: instance of sageImage
    
    returns: list of matrices [R,G,B] representing the monochromatic components of the image

Mann kann ein Bild sehen wie eine $m \times n$ Matrix, wobei $m$ die Höhe in Pixels ist und $n$ die Breite. Die Einträge der Matrix sind dann $3$-Tupel die die Rot-, Grün- und Blauwerte derstellen. Also die Einträge sind der Form $(R,G,B) \in \{0,\dots , 255\}^3$. Sage denkt leider nicht so über Bilder, deswegen müssen wir sie helfen es richtig zu sehen. Diese Funktion nimmt wie ihr Argument ein `sageImage`, das ist einfach ein Bild, das schon vom Programm ausgelsen und bearbeitungsfähig ist. Sie trennt dann die Rot-, Grün- und Blauwerte jedes Pixels und bewahrt diese Zahl in der zugehörige Matrix, von der Sage versteht, dass es eine Matrix ist. Am Ende werden also drei Matrizen retourniert, eine für rot, eine für grün und eine für blau.

## Von Matrizen nach ein Bild

Synopsis:

    matricestoimage(matrices, filename, saveimage, showimage)
    
    matrices: list of matrices [R,G,B] of equal size representing the colourencodings for red, geen and blue
    
    filename: name to be given to the file that the image is saved as
    default:  'untitled.png'
    
    saveimage: boolean indicating wether or not to save the file in the directory of this notebook
    default: False
    
    showimage: boolean indicating wether or not to render the synthesized image
    default: False
    
    returns: image as an instance of sageImage
   
Wenn Sage einmal verstanden hat, dass es tatsächlich um Dreitupel der Matizen geht hier, hat sie vergessen wie sie davon zürück ein Bild fromen kann. Diese Fuktion tut also genau dass. Sie nimmt eine Liste von drei Matrizen wie ihr Argument und bildet davon wieder ein Bild. Nachher kann sie dieses Bild bewharen, dafür muss `saveimage=True` gespeichert werden. In diesem Fall sollte mann auch eine Name speichern, die dem bewahrte Bild gegeben werden sollte. Das tut mann mittels `filename='beliegigename.png'.` Letztens kann die Funktion das geformte Bild auch zeigen mit `showimage=True`.    

## Von einer Datei nach Matrizen

synopsis:

    filetomatrices(filename)
    
    filename: string specifying the file name or the path if the file is not in the same directory as this notebook
    
    returns: list of matrices [R,G,B] representing the monochromatic components of the image
    
Diese Funktion ist sehr ähnlich zu `imagetometrices`, aber hier ist das Argument nicht ein Objekt , intern zum Prigramm, sondern eine Name die eine Datei extern zum Programm identifizeirt. So können wir auch externe Dateien auslesen und ins Programm benutzen.

## Bilder komprimieren

Synopsis:
    
    compressimage(image, k, filename, showresult, saveresult)
    
    matrices: list of matrices [R,G,B] of equal size representing the colourencodings for red, geen and blue
    
    k: number of singular values that are to be used (together with their left and right singular vectors)
    
    filename: name to be given to the file that the image is saved as
    default:  'untitled_compressed.png'
    
    saveresult: boolean indicating wether or not to save the compressed file in the directory of this notebook
    default: False
    
    showresult: boolean indicating wether or not to render the compressed image
    default: False
    
    returns: compressed image as an instance of sageImage

Diese Funktion komprimiert ein Bild mithilfe der Singularwertzerlegung. Sie nimmt ein Bild wie ihr erstes Argument und eine ganze Zahls `k` wie ihr zweites und trennt das Bild in drei Matrizen, deren ersten `k` Singularwerten und Linker- und Rechtersingularvektoren eriinert werden. Alle übrigen können vergessen werden, und mit nur dieser Information wird dann wieder ein Bild gebildet, was mann bewahren und zeigen lassen kann, genau so wie in `martricestoimage`.


In [ ]:
# Beispiel

image = compressimage('goat.png', 20, 'goat_compressed20svs.png', True, True)

In [ ]:
# Beispiel

image = compressimage('goat.png', 100)

image.show()

In [ ]:
# Beispiel

image = compressimage('goat.png', 4)

image.save('goat_compressed_severe_almost_unrecognisable.png')